## Carga modulo comun

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../common'))

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from purrfect.dataset import load_partition,save_partition, create_train_valid_loaders

from purrfect.training import train_model
import torch.optim as optim

from purrfect.active_learning import create_new_partition, test_model

from sklearn.model_selection import train_test_split
#from purrfect.submission import create_submission

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

## Definición modelo

In [4]:
# Define the double convolution block
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class ChannelAdder(nn.Module):
    def __init__(self):
        super(ChannelAdder, self).__init__()
        # Define Sobel and Laplacian kernels as tensors
        self.sobel_x = torch.tensor([[-1., 0., 1.],
                                     [-2., 0., 2.],
                                     [-1., 0., 1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)
        
        self.sobel_y = torch.tensor([[-1., -2., -1.],
                                     [ 0.,  0.,  0.],
                                     [ 1.,  2.,  1.]], dtype=torch.float32,device=DEVICE).unsqueeze(0).unsqueeze(0)

    def forward(self, x):
        epsilon = 1e-8

        # Extract the first, second, and third channels
        first_channel = x[:, 0, :, :].unsqueeze(1)  # e1 (first channel)
        second_channel = x[:, 1, :, :].unsqueeze(1)  # e2 (second channel)
        third_channel = x[:, 2, :, :].unsqueeze(1)  # error (third channel)

        # 1. Compute the first new channel: sqrt(first_channel^2 + second_channel^2)
        new_channel1 = torch.sqrt(first_channel**2 + second_channel**2)

        # 2. Compute the second new channel: 1/2 * arctan(channel2 / channel1)
        new_channel2 = 0.5 * torch.atan(second_channel / (first_channel + epsilon))

        # 3. Compute Sobel gradients and Laplacians for e1 (first_channel)
        grad_e1_x = F.conv2d(first_channel, self.sobel_x, padding=1)
        grad_e1_y = F.conv2d(first_channel, self.sobel_y, padding=1)
        grad_e1_magnitude = torch.sqrt(grad_e1_x**2 + grad_e1_y**2)

        # 4. Compute Sobel gradients and Laplacians for e2 (second_channel)
        grad_e2_x = F.conv2d(second_channel, self.sobel_x, padding=1)
        grad_e2_y = F.conv2d(second_channel, self.sobel_y, padding=1)
        grad_e2_magnitude = torch.sqrt(grad_e2_x**2 + grad_e2_y**2)

        # 5. Compute weighted ellipticity channels (e1_weighted, e2_weighted)
        #e1_weighted = first_channel / (third_channel + epsilon)
        #e2_weighted = second_channel / (third_channel + epsilon)

        # Concatenate all the channels (original and new) into the output tensor
        output = torch.cat([
            x,                 # Original 3 channels
            new_channel1,      # sqrt(channel1^2 + channel2^2)
            new_channel2,      # 1/2 * arctan(channel2 / channel1)
            #grad_e1_x,         # Gradient X of channel1
            #grad_e1_y,         # Gradient Y of channel1
            #grad_e2_x,         # Gradient X of channel2
            #grad_e2_y,         # Gradient Y of channel2
            grad_e1_magnitude, # Gradient magnitude of channel1
            grad_e2_magnitude, # Gradient magnitude of channel2
            #laplacian_e1,      # Laplacian of channel1
            #laplacian_e2,      # Laplacian of channel2
            #e1_weighted,       # e1_weighted
            #e2_weighted        # e2_weighted
        ], dim=1)

        return output

# Define the U-Net model
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        # Channel Adder
        self.channel_adder = ChannelAdder()
        # Encoder
        self.encoder1 = DoubleConv(in_channels+4, 64)
        self.encoder2 = DoubleConv(64, 128)
        self.encoder3 = DoubleConv(128, 256)
        self.encoder4 = DoubleConv(256, 512)

        # Bottleneck
        self.bottleneck = DoubleConv(512, 1024)

        # Decoder
        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.decoder4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.decoder3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.decoder2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.decoder1 = DoubleConv(128, 64)

        # Final output layer
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Agregar features
        x = self.channel_adder(x)
        
        # Encoder
        e1 = self.encoder1(x)
        
        e2 = self.encoder2(F.max_pool2d(e1, 2))
        e3 = self.encoder3(F.max_pool2d(e2, 2))
        e4 = self.encoder4(F.max_pool2d(e3, 2))

        # Bottleneck
        b = self.bottleneck(F.max_pool2d(e4, 2))
        # Decoder with skip connections
        d4 = self.upconv4(b)
        d4 = torch.cat((e4, d4), dim=1)
        d4 = self.decoder4(d4)

        d3 = self.upconv3(d4)
        d3 = torch.cat((e3, d3), dim=1)
        d3 = self.decoder3(d3)

        d2 = self.upconv2(d3)
        d2 = torch.cat((e2, d2), dim=1)
        d2 = self.decoder2(d2)

        d1 = self.upconv1(d2)
        d1 = torch.cat((e1, d1), dim=1)
        d1 = self.decoder1(d1)

        # Final output
        out = self.final_conv(d1)
        return out

## Creación particion inicial

In [5]:
def create_next_partitions(current_partition):#Creacion de particiones train y valid
    init_partition = load_partition(f"partition_{2*current_partition+1}.json")
    init_partition += load_partition(f"partition_{2*current_partition+2}.json")
    train_partition, val_partition = train_test_split(init_partition, test_size=0.2, random_state=42)
    save_partition(f"partition_{current_partition+1}_train.json","partitions",train_partition)
    save_partition(f"partition_{current_partition+1}_val.json","partitions",val_partition)

In [6]:
create_next_partitions(0)

## Carga modelo

In [7]:
#Define model
model = UNet( 3, 1)
model = model.to(DEVICE)
# Define Loss
criterion = torch.nn.L1Loss()

In [8]:
current_partition = 1
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)

In [9]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=BATCH_SIZE,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:17<00:00,  4.37it/s, WMAPE=3.91, DICE=0.233, DPEAKS=114, Loss=0.0181]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:16<00:00,  4.57it/s, WMAPE=5.19, DICE=0.14, DPEAKS=104, Loss=0.0157]


Saving best model
Epoch [3/50]


...

Epoch [18/50]


Validate Epoch 18: 100%|██████████| 77/77 [00:16<00:00,  4.70it/s, WMAPE=4.3, DICE=0.0559, DPEAKS=43.1, Loss=0.0122]


Epoch [19/50]


Validate Epoch 19: 100%|██████████| 77/77 [00:15<00:00,  4.99it/s, WMAPE=1.85, DICE=0.0543, DPEAKS=40.4, Loss=0.00596]


early stopping: 3 epochs without improvement
Training complete.


In [10]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)

Validate Epoch test: 100%|██████████| 971/971 [03:29<00:00,  4.63it/s, WMAPE=1.89, DICE=0.0538, DPEAKS=40.7, Loss=0.0057]


In [11]:
create_next_partitions(current_partition)

In [12]:
current_partition = 1
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
model.load_state_dict(torch.load(best_model_path,weights_only=True))
optimizer = optim.Adam(model.parameters())


In [13]:
current_partition=2
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=BATCH_SIZE,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:13<00:00,  5.81it/s, WMAPE=4.07, DICE=0.0575, DPEAKS=39, Loss=0.0121]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:13<00:00,  5.77it/s, WMAPE=3.13, DICE=0.0565, DPEAKS=43.7, Loss=0.00952]


Saving best model
Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:14<00:00,  5.29it/s, WMAPE=2.94, DICE=0.0585, DPEAKS=45.8, Loss=0.00939]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:14<00:00,  5.23it/s, WMAPE=5.61, DICE=0.0714, DPEAKS=64.4, Loss=0.0174]


early stopping: 3 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
optimizer = optim.Adam(model.parameters())
#optimizer.load_state_dict(torch.load(last_checkpoint_path,weights_only=True)["optimizer"])
test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 971/971 [03:04<00:00,  5.28it/s, WMAPE=1.82, DICE=0.0536, DPEAKS=39, Loss=0.00555]


In [ ]:
for current_partition in range(3,9):
    train_loader, val_loader = create_train_valid_loaders(
        f"partition_{current_partition}_train.json",
        f"partition_{current_partition}_val.json",
        "partitions",
        batch_size=BATCH_SIZE,
    )
    best_model_path = os.path.join(
        "models", f"best_model_partition_{current_partition}.pth"
    )
    last_checkpoint_path = os.path.join(
        "models", f"last_checkpoint_partition_{current_partition}.pth"
    )
    train_model(
        model,
        train_loader,
        val_loader,
        best_model_path,
        last_checkpoint_path,
        criterion,
        optimizer,
        num_epochs=50,
        device=DEVICE,
        early_stopping_patience=3,
    )
    model.load_state_dict(torch.load(best_model_path,weights_only=True))
    optimizer = optim.Adam(model.parameters())
    optimizer.load_state_dict(torch.load(last_checkpoint_path,weights_only=True)["optimizer"])
    test_model(model,criterion,device=DEVICE,batch_size=BATCH_SIZE)
    create_next_partitions(current_partition)

In [19]:
current_partition=3
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)

In [20]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
optimizer = optim.Adam(model.parameters())
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  5.96it/s, WMAPE=2.03, DICE=0.0603, DPEAKS=61.1, Loss=0.00608]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.13it/s, WMAPE=5.36, DICE=0.0579, DPEAKS=45, Loss=0.0157]


Epoch [3/50]


...

Epoch [5/50]


Validate Epoch 5: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.04, DICE=0.0634, DPEAKS=56.6, Loss=0.00611]


Epoch [6/50]


Validate Epoch 6: 100%|██████████| 77/77 [00:10<00:00,  7.17it/s, WMAPE=2.73, DICE=0.0758, DPEAKS=71.2, Loss=0.0079]


early stopping: 3 epochs without improvement
Training complete.


In [21]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)

Validate Epoch test: 100%|██████████| 1941/1941 [04:54<00:00,  6.59it/s, WMAPE=1.77, DICE=0.0499, DPEAKS=38.4, Loss=0.00553]


TypeError: create_next_partitions() missing 2 required positional arguments: 'model' and 'criterion'

In [23]:
create_next_partitions(current_partition)

In [24]:
current_partition=4

In [25]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:12<00:00,  6.06it/s, WMAPE=1.51, DICE=0.0393, DPEAKS=27.1, Loss=0.00439]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:11<00:00,  6.91it/s, WMAPE=1.72, DICE=0.0389, DPEAKS=28.4, Loss=0.00493]


Epoch [3/50]


...

Epoch [8/50]


Validate Epoch 8: 100%|██████████| 77/77 [00:10<00:00,  7.07it/s, WMAPE=1.45, DICE=0.0379, DPEAKS=27.2, Loss=0.00424]


Epoch [9/50]


Validate Epoch 9: 100%|██████████| 77/77 [00:11<00:00,  6.50it/s, WMAPE=1.41, DICE=0.038, DPEAKS=26.9, Loss=0.00424]


early stopping: 3 epochs without improvement
Training complete.


In [26]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition)

Validate Epoch test: 100%|██████████| 1941/1941 [04:50<00:00,  6.68it/s, WMAPE=1.41, DICE=0.0383, DPEAKS=28.1, Loss=0.00421]


In [27]:
current_partition=5

In [10]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  7.86it/s, WMAPE=1.87, DICE=0.0453, DPEAKS=46.4, Loss=0.00513]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:10<00:00,  7.59it/s, WMAPE=1.97, DICE=0.0453, DPEAKS=44, Loss=0.00521]


Epoch [3/50]


Validate Epoch 3: 100%|██████████| 77/77 [00:09<00:00,  8.01it/s, WMAPE=2.45, DICE=0.0461, DPEAKS=44, Loss=0.00671]


Epoch [4/50]


Validate Epoch 4: 100%|██████████| 77/77 [00:09<00:00,  7.87it/s, WMAPE=1.92, DICE=0.0455, DPEAKS=45.8, Loss=0.00514]


early stopping: 3 epochs without improvement
Training complete.


In [11]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_6_val: 100%|██████████| 77/77 [00:10<00:00,  7.21it/s, WMAPE=1.65, DICE=0.0436, DPEAKS=36.1, Loss=0.00477]


In [12]:
current_partition=6

In [13]:
train_loader, val_loader = create_train_valid_loaders(
    f"partition_{current_partition}_train.json",
    f"partition_{current_partition}_val.json",
    "partitions",
    batch_size=16,
)
best_model_path = os.path.join(
    "models", f"best_model_partition_{current_partition}.pth"
)
last_checkpoint_path = os.path.join(
    "models", f"last_checkpoint_partition_{current_partition}.pth"
)
optimizer = optim.Adam(model.parameters(), lr=0.00001)
train_model(
    model,
    train_loader,
    val_loader,
    best_model_path,
    last_checkpoint_path,
    criterion,
    optimizer,
    num_epochs=50,
    device=DEVICE,
    early_stopping_patience=3,
)

Epoch [1/50]


Validate Epoch 1: 100%|██████████| 77/77 [00:09<00:00,  8.14it/s, WMAPE=2.05, DICE=0.046, DPEAKS=52.4, Loss=0.00538]


Saving best model
Epoch [2/50]


Validate Epoch 2: 100%|██████████| 77/77 [00:09<00:00,  8.04it/s, WMAPE=2.06, DICE=0.0462, DPEAKS=52.8, Loss=0.00534]


Saving best model
Epoch [3/50]


...

Epoch [14/50]


Validate Epoch 14: 100%|██████████| 77/77 [00:09<00:00,  8.05it/s, WMAPE=2.14, DICE=0.0452, DPEAKS=49.8, Loss=0.00559]


Epoch [15/50]


Validate Epoch 15: 100%|██████████| 77/77 [00:09<00:00,  8.12it/s, WMAPE=2.04, DICE=0.046, DPEAKS=50.1, Loss=0.00524]


early stopping: 3 epochs without improvement
Training complete.


In [14]:
#Cargar mejor modelo de la particion actual
model.load_state_dict(torch.load(best_model_path,weights_only=True))
test_model(model,criterion,device=DEVICE,batch_size=16)
create_next_partitions(current_partition,model,criterion,device=DEVICE)

Validate Epoch partition_7_val: 100%|██████████| 77/77 [00:10<00:00,  7.23it/s, WMAPE=1.54, DICE=0.043, DPEAKS=33.5, Loss=0.00457]
